# trajectory_analyze

particle trajectory analysis

files used:
- particle netcdf file generated by parcels after simulation
- the actual ocean data the particles ran on (if you want to use HFR data in your analysis or something idk)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.spatial
import xarray as xr

import pyplume.utils as utils
from pyplume.constants import *
from pyplume.dataloaders import SurfaceGrid, open_dataset
from pyplume.postprocess import *

In [ ]:
# the data generated by ParticleFile
p_nc = utils.FILES_ROOT / utils.PARTICLE_NETCDF_DIR / "particle_hunington_spill_oct.nc"
# how to open the particle data
p_ds = open_dataset(p_nc)
# p_ds is closed, but can still be used
p_ds.close()
p_results = ParticleResult(p_ds)

idk you can do some shenanigans with the particle result

In [ ]:
from pyplume.plotting import draw_trajectories_datasets
domain = {
    "S": 33,
    "N": 34,
    "W": -118.5,
    "E": -117.7
}
draw_trajectories_datasets([p_results.ds], ["hello"], domain=domain, legend=False)

In [ ]:
plt.plot(p_results.data_vars["buoy_distances"][0])